# Questrade Portfolio Manager

In [2]:
import os
import math
import requests
import momentum
import pyticker
import dividend
import questrade
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from qtrade import Questrade as qt
import pandas_datareader.data as web
#from questrade_api import Questrade

## Balance Summary

In [3]:
balance = questrade.get_balance_summary()
balance

{'currency': 'CAD', 'cash': 31785.3365, 'marketValue': 0, 'totalEquity': 31785.3365, 'buyingPower': 31785.3365, 'maintenanceExcess': 31785.3365, 'isRealTime': True}
{'currency': 'USD', 'cash': 21649.7415, 'marketValue': 10340.8, 'totalEquity': 31990.5415, 'buyingPower': 21649.7415, 'maintenanceExcess': 21649.7415, 'isRealTime': True}


,Cash,Market_Value,Total_Equity
Currency,,,
CAD,31785.3365,0.0,31785.3365
USD,21649.7415,10340.8,31990.5415


## Position Summary

In [ ]:
position_data = {
    'Symbol': [],
    'Description': [],
    'Currency': [],
    'Quantities': [],
    'Market Value': [],
    'Gain/Loss (%)': []
}
total_costs = 0
total_market_value = 0
for account in acctId:
    positions = qtrade.get_account_positions(account)
    for position in positions:
        symbol = position['symbol']
        description = qtrade.ticker_information(symbol)['description']
        qty = position['openQuantity']
        cmv = position['currentMarketValue']
        currency = qtrade.ticker_information(symbol)['currency']
        cost = position['totalCost']
        change = round(100 * (cmv - cost) / cost, 2)

        total_market_value = total_market_value + cmv
        total_costs = total_costs + cost
        position_data['Symbol'].append(symbol)
        position_data['Description'].append(description)
        position_data['Currency'].append(currency)
        position_data['Quantities'].append(qty)
        position_data['Market Value'].append(cmv)
        position_data['Gain/Loss (%)'].append(change)

In [ ]:
portfolio = pd.DataFrame(position_data)
portfolio.set_index('Symbol', inplace=True)
portfolio

In [ ]:
total_equity = balance['perCurrencyBalances'][1]['totalEquity']
portfolio['%Portfolio'] = [
    round(100 * (x / total_equity), 2) for x in portfolio['Market Value']
]

In [ ]:
portfolio

In [ ]:
portfolio['%Portfolio'].plot(kind='pie',
                             title='Portfolio Breakdown',
                             figsize=(10, 10),
                             legend=True)

## Account Balance Summary

In [ ]:
balance_summary = {
    'Category':
    ['Cash', 'Market Value', 'Total Equity', 'Cash (%)', 'Equity (%)'],
    'CAD': [],
    'USD': []
}

for currency in balance['perCurrencyBalances']:
    if currency['currency'] == 'CAD':
        balance_summary['CAD'].append(currency['cash'])
        balance_summary['CAD'].append(currency['marketValue'])
        balance_summary['CAD'].append(currency['totalEquity'])
        balance_summary['CAD'].append(
            round(100 * currency['cash'] / currency['totalEquity'], 2))
        balance_summary['CAD'].append(
            round(100 * currency['marketValue'] / currency['totalEquity'], 2))
    else:
        balance_summary['USD'].append(currency['cash'])
        balance_summary['USD'].append(currency['marketValue'])
        balance_summary['USD'].append(currency['totalEquity'])
        balance_summary['USD'].append(
            round(100 * currency['cash'] / currency['totalEquity'], 2))
        balance_summary['USD'].append(
            round(100 * currency['marketValue'] / currency['totalEquity'], 2))

# balance_summary

balance_df = pd.DataFrame(balance_summary)
balance_df.set_index("Category", inplace=True)
balance_df

## Monthly Dividend Income

In [ ]:
startDate = '2018-04-01'
endDate = dt.date.today().strftime("%Y-%m-%d")
dtrange = pd.date_range(startDate, endDate, freq='d')
months = pd.Series(dtrange.month)
starts, ends = months.ne(months.shift(1)), months.ne(months.shift(-1))
startEndDates = pd.DataFrame({
    'month_starting_date':
    dtrange[starts].strftime('%Y-%m-%d'),
    'month_ending_date':
    dtrange[ends].strftime('%Y-%m-%d')
})
dateList = startEndDates.values.tolist()

In [ ]:
output = {}
total_div_earned = 0

for date in dateList:
    start = date[0]
    end = date[1]
    activities = qtrade.get_account_activities(acctId[0], start, end)
    monthly_div = 0
    for activity in activities:
        if activity['type'] == 'Dividends':
            monthly_div = monthly_div + activity['netAmount']
    output[dt.datetime.strptime(start,
                                "%Y-%m-%d").strftime("%Y-%m")] = monthly_div
    total_div_earned = total_div_earned + monthly_div

In [ ]:
monthly_div_df = pd.DataFrame.from_dict(output,
                                        orient='index',
                                        columns=['Monthly_Dividend_Income'])
monthly_div_df

## Dividend Income History with Bar Chart Visualization

In [ ]:
monthly_div_df.plot(kind='bar',
                    title='Monthly Dividend Payments',
                    xlabel='Months',
                    ylabel='Dividend ($)',
                    figsize=(15, 10),
                    legend=False,
                    ylim=(0, 30))

## Portfolio Performance

In [ ]:
performance_summary = {
    'Portfolio Return (%)': [],
    'Total Dividend Earned ($)': []
}

performance_summary['Portfolio Return (%)'].append(
    100 * (total_market_value - total_costs) / total_costs)
performance_summary['Total Dividend Earned ($)'].append(total_div_earned)
performance_df = pd.DataFrame(performance_summary)
performance_df

In [ ]:
portfolio

In [ ]:
def color_negative_red(value):

    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'

    return 'color: %s' % color

In [ ]:
portfolio = portfolio[['Description', 'Currency', '%Portfolio', 'Gain/Loss (%)']]
portfolio = portfolio.style.applymap(color_negative_red, subset=['Gain/Loss (%)'])

In [ ]:
portfolio

In [ ]:
balance_df

In [ ]:
balance_df.loc['Total Equity', 'USD']

In [ ]:
def get_latest_price(symbol: str):
    startDate = (dt.date.today() -
                 dt.timedelta(days=1)).strftime("%Y-%m-%d")
    endDate = dt.date.today().strftime("%Y-%m-%d")
    price_data = web.DataReader(symbol, 'yahoo', startDate, endDate)
    return price_data['Adj Close'][-1]


def calculate_shares(portion: float, symbol: str):
    total_equity = balance_df.loc['Total Equity', 'USD']
    amount = total_equity * portion
    curr_price = get_latest_price(symbol)
    return (amount, math.floor(amount / curr_price))

In [ ]:
get_latest_price('MMM')

In [ ]:
calculate_shares(0.03, 'MO')

In [ ]:
calculate_shares(0.03, 'JPM')